<a href="https://colab.research.google.com/github/PrinceWallyy/CV_DeepLearning/blob/master/FINAL_Computervision_Assignment_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocess

In [25]:
from google.colab import drive
drive.mount("/content/drive")
path = '/content/drive/MyDrive/MU/Computer Vision/Assignment 2 - CNN/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
%%capture
!pip install wandb -q

In [27]:
import wandb
from wandb.keras import WandbCallback

# WandB – Login to your wandb account so you can log all your metrics
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [28]:
#load libaries
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os


# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
np.random.seed(42)
tf.random.set_seed(42)

In [29]:
#emotion map
emotion_map = {
    0:'anger',
    1:'disgust',
    2:'fear', 
    3:'happiness', 
    4: 'sadness', 
    5: 'surprise', 
    6: 'neutral'}

# Data

In [30]:
#load data
data = pd.read_csv(path+'fer2013.csv', delimiter=',')

In [31]:
 df_train = data[data["Usage"]=="Training"]
 df_test_public = data[data["Usage"]=="PublicTest"]
 df_test_private = data[data["Usage"]=="PrivateTest"] 

## Format training set:

In [32]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage import io, transform
import torchvision
import torch

#get image size
image_size = len(df_train['pixels'].iloc[0].split(' '))
width = int(math.sqrt(image_size))
height = int(math.sqrt(image_size))

#reshape image, and add dimension for CNN 
img_features = df_train['pixels'].apply(lambda x: np.array(x.split()).reshape(height, width, 1).astype('float32'))
img_features = np.stack(img_features, axis=0)
img_features = img_features / 255.0
#remove index from label
img_labels =  np.array(df_train['emotion'])

#shuffle and split into train and test set:
X_train, X_valid, y_train, y_valid = train_test_split(img_features, img_labels,
                                                      shuffle=True, stratify=img_labels,
                                                      test_size=0.3, random_state=42)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(20096, 48, 48, 1) (8613, 48, 48, 1) (20096,) (8613,)


## Format testing set:

In [33]:
#reshape image, and add dimension for CNN 
img_features = df_test_private['pixels'].apply(lambda x: np.array(x.split()).reshape(height, width, 1).astype('float32'))
img_features = np.stack(img_features, axis=0)
img_features = img_features / 255.0

#remove index from label
img_labels =  np.array(df_test_private['emotion'])
X_private_test = img_features
y_private_test = img_labels

#reshape image, and add dimension for CNN 
img_features = df_test_public['pixels'].apply(lambda x: np.array(x.split()).reshape(height, width, 1).astype('float32'))
img_features = np.stack(img_features, axis=0)
img_features = img_features / 255.0

#remove index from label
img_labels =  np.array(df_test_public['emotion'])
X_public_test = img_features
y_public_test = img_labels

# Training

## Weights and Biases Sweep
Here a Weights and Biases sweep gets defined (only run once to initalize).
The worker then takes care of using the right parameters.


In [ ]:
import wandb

sweep_config = {
  "name" : "CNN_Bayes_optimization",
  #Search method
  "method" : "bayes",
  "metric":{
    "name": "val_loss",
    "goal": "minimize"
  },
  "parameters" : {
    "epochs" : {
      "values" : [15]
    },
    "learning_rate" :{
      "values" : [0.0005]
    },
    "layer_1_filter" :{
      "values" :[64]
    },
    "layer_2_filter" :{
      "values" :[256]
    },
    "layer_3_filter" :{
      "values" :[256]
    },
    "layer_4_filter" :{
      "values" :[512]
    },
    "dropout" :{
      "values" :[0.25]
    },
    "batchnorm" :{
      "values" :[True]
    },
    "batch_size": {
        "values": [64]
    },
    "loss_function": {
        "values": ["sparse_categorical_crossentropy"]
    }
    

  }
}

sweep_id = wandb.sweep(sweep_config,
                        entity="mu_cv_cnn",
                        project="mu-cv-assignment2")

Create sweep with ID: 6llkfa0j
Sweep URL: https://wandb.ai/mu_cv_cnn/mu-cv-assignment2/sweeps/6llkfa0j


In [13]:
#To use if you want to run a differnt sweep: 
#sweep_id = "6llkfa0j"

In [14]:
import gc
# Custom Callback To Include in Callbacks List At Training Time
# collect garbage to reduce RAM usage
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [15]:
#Define mode
def make_model(config):
  # WandB Config managed by sweeps 
  model = models.Sequential()

  # First convolutional block:
  model.add(layers.Conv2D(config.layer_1_filter, (3,3), padding='same', input_shape=(48,48,1)))
  # if batchnorm is activated
  if config.batchnorm:
    model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  if config.dropout > 0:
    model.add(layers.Dropout(config.dropout))

  # Second convolutional block:
  model.add(layers.Conv2D(config.layer_2_filter,(5,5), padding='same'))
  if config.batchnorm:
    model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  if config.dropout > 0:
    model.add(layers.Dropout(config.dropout))

  
  # Third convolutional block:
  model.add(layers.Conv2D(config.layer_3_filter,(3,3), padding='same'))
  if config.batchnorm:
    model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  if config.dropout > 0:
    model.add(layers.Dropout(config.dropout))

  # Fourth convolutional block:
  model.add(layers.Conv2D(config.layer_4_filter,(3,3), padding='same'))
  if config.batchnorm:
    model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  if config.dropout > 0:
    model.add(layers.Dropout(config.dropout))

  model.add(layers.Flatten())

  # First fully connected Block
  model.add(layers.Dense(256))
  if config.batchnorm:
    model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  if config.dropout > 0:
    model.add(layers.Dropout(config.dropout))

  # Second fully connected Block
  model.add(layers.Dense(512))
  if config.batchnorm:
    model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  if config.dropout > 0:
    model.add(layers.Dropout(config.dropout))
  #Output layer
  model.add(layers.Dense(7, activation='softmax'))
  return model

In [24]:
#convert label number to emotion, for plotting
y_emotion = [emotion_map[label] for label in y_valid]

In [20]:
#define train method
def train():
    with wandb.init() as run:
        #get config
        config = wandb.config
        #make model
        model = make_model(config)
        optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
        model.compile(optimizer, config.loss_function, metrics=['acc'])

        loss = model.fit(X_train, y_train, epochs=config.epochs, 
                          validation_data=(X_valid, y_valid),
                          batch_size = config.batch_size,
                          #log weights, gradients, images and model
                          callbacks=[WandbCallback(log_weights=True,data_type='image',labels=y_emotion,save_model=True,log_gradients=True,save_graph=True,training_data=(X_train,y_train),validation_data=(X_valid,y_valid)),GarbageCollectorCallback()])
        #Evaluate on private test
        private_loss, private_accuracy = model.evaluate(X_private_test, y_private_test)
        wandb.log({'private_test_accuracy': private_accuracy,
                   'private_test_loss': private_loss
                   })

        #Evaluate on public test
        public_loss, public_accuracy = model.evaluate(X_public_test, y_public_test)
        wandb.log({'public_test_accuracy': public_accuracy,
                   'public_test_loss': public_loss
                   })
        predictions = model.predict(X_public_test)
        #log confusion matrix
        wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                        y_true=y_public_test, preds=predictions,
                        class_names=['anger','disgust','fear','happiness','sadness','surprise','neutral'])})
        
count = 1 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count,
                        entity="mu_cv_cnn",
                        project="mu-cv-assignment2")

wandb: Agent Starting Run: 3vhb7znl with config:
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: 0.25
wandb: 	epochs: 15
wandb: 	layer_1_filter: 64
wandb: 	layer_2_filter: 256
wandb: 	layer_3_filter: 256
wandb: 	layer_4_filter: 512
wandb: 	learning_rate: 0.0005
wandb: 	loss_function: sparse_categorical_crossentropy


Epoch 1/15
314/314 [==============================] - 22s 67ms/step - loss: 2.0647 - acc: 0.2244 - val_loss: 1.9963 - val_acc: 0.1939
Epoch 2/15
314/314 [==============================] - 21s 67ms/step - loss: 1.6666 - acc: 0.3634 - val_loss: 1.5825 - val_acc: 0.3876
Epoch 3/15
314/314 [==============================] - 21s 67ms/step - loss: 1.4693 - acc: 0.4316 - val_loss: 1.3334 - val_acc: 0.4887
Epoch 4/15
314/314 [==============================] - 21s 67ms/step - loss: 1.3268 - acc: 0.4942 - val_loss: 1.3005 - val_acc: 0.4947
Epoch 5/15
314/314 [==============================] - 21s 67ms/step - loss: 1.2382 - acc: 0.5291 - val_loss: 1.2465 - val_acc: 0.5197
Epoch 6/15
314/314 [==============================] - 21s 67ms/step - loss: 1.1834 - acc: 0.5490 - val_loss: 1.1967 - val_acc: 0.5388
Epoch 7/15
314/314 [==============================] - 21s 67ms/step - loss: 1.1266 - acc: 0.5664 - val_loss: 1.1430 - val_acc: 0.5682
Epoch 8/15
314/314 [==============================] - 21s 67ms

epoch,14
loss,0.82043
acc,0.69044
val_loss,1.12649
val_acc,0.58365
_runtime,891
_timestamp,1621944123
_step,16
best_val_loss,1.0694
best_epoch,13
private_test_accuracy,0.59682


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▅▄▄▃▃▃▂▂▂▂▁▁▁
acc,▁▃▄▅▅▆▆▆▇▇▇▇███
val_loss,█▅▃▃▂▂▂▂▄▁▂▁▁▁▁
val_acc,▁▄▆▆▇▇▇▇▅█▇████
_runtime,▁▂▂▃▃▃▄▅▅▆▆▆▇▇███
_timestamp,▁▂▂▃▃▃▄▅▅▆▆▆▇▇███
_step,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
private_test_accuracy,▁
private_test_loss,▁
public_test_accuracy,▁


Run 3vhb7znl errored: ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')
wandb: ERROR Run 3vhb7znl errored: ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')


# Old code for Grid search

In [ ]:
#params:
import gc
learning_rates = [0.0005]
epochs = 15
layer_1_filter = [64,128]
layer_2_filter = [128,256]
layer_3_filter = [256,512]
layer_4_filter = [512,1028]
dropout = [0,0.25,0.5]
batchnorm = [True,False]
number_runs = 0
lz = 16

for lr in learning_rates:
  for l1 in layer_1_filter:
    for l2 in layer_2_filter:
      for l3 in layer_3_filter:
        for l4 in layer_4_filter:
          for bn in batchnorm:
            for dr in dropout:
              tf.keras.backend.clear_session()
              if number_runs < 10:
                number_runs += 1
                continue
              run = wandb.init(project='MU-CV-Assignment2',
                              config={  # and include hyperparameters and metadata
                                  "learning_rate": lr,
                                  "epochs": epochs,
                                  "batch_size": 64,
                                  "loss_function": "sparse_categorical_crossentropy",
                                  "architecture": "CNN-4-Layers+",
                                  "dataset": "FER-2013",
                                  "layer_1_filter": l1,
                                  "layer_2_filter": l2,
                                  "layer_3_filter": l3,
                                  "layer_4_filter": l4,
                                  "layers": lz,
                                  "dropout": dr,
                                  "batchnorm": bn
                              },
                              entity="mu_cv_cnn")
              #change name of the run to be more speaking:
              wandb.run.name = "C_"+str(l1)+"_"+str(l2)+"_"+str(l3)+"_"+str(l4)+"_D"+"256_512"+"_DR_"+str(dr)+"_BN_"+str(bn)+"_"+str(number_runs)
              #wandb.run.save()
              config = wandb.config  # We'll use this to configure our experiment


              


              # Compile model like you usually do.
              # Notice that we use config, so our metadata matches what gets executed
              optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
              model.compile(optimizer, config.loss_function, metrics=['acc'])

              history = model.fit(X_train, y_train, epochs=config.epochs, 
                                  validation_data=(X_valid, y_valid),
                                  batch_size = config.batch_size,
                                  callbacks=[WandbCallback(),GarbageCollectorCallback()])
              # Dont save images
              #                   callbacks=[WandbCallback(data_type="image", labels=y_train)])

              loss, accuracy = model.evaluate(X_valid, y_valid)
              print('Test Error Rate: ', round((1 - accuracy) * 100, 2))

              # With wandb.log, we can easily pass in metrics as key-value pairs.
              wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})

              run.join()
              number_runs += 1
              del model
              del history
              gc.collect()

In [ ]:
wandb.init(job_type="analysis")

# Code for plotting the comparison of different search strategies

In [ ]:
import wandb
api = wandb.Api()
runs = api.runs("mu_cv_cnn/mu-cv-assignment2")
summary_list = [] 
config_list = [] 
name_list = [] 
for run in runs: 
    # run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.  We remove special values that start with _.
    config_list.append({k:v for k,v in run.config.items()}) 

    # run.name is the name of the run.
    name_list.append(run.name)       

import pandas as pd 
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
all_df = pd.concat([name_df, config_df,summary_df], axis=1)

In [ ]:
all_df.to_csv(path+"project.csv")

In [ ]:
all_df.columns

Index(['name', 'epochs', 'dropout', 'batchnorm', 'batch_size', 'learning_rate',
       'loss_function', 'layer_1_filter', 'layer_2_filter', 'layer_3_filter',
       'layer_4_filter', 'layers', 'dataset', 'architecture', 'dropout_value',
       'layer_1_filters', 'layer_2_filters', 'layer_3_filters',
       'layer_4_filters', 'layer_5_filter', 'gradients/dense/bias.gradient',
       'parameters/conv2d.weights', 'parameters/conv2d_1.weights',
       'gradients/conv2d_3/kernel.gradient', 'parameters/conv2d.bias',
       'parameters/dense_2.bias', 'gradients/dense_1/bias.gradient',
       'gradients/batch_normalization/gamma.gradient', 'public_test_loss',
       'gradients/batch_normalization_5/gamma.gradient',
       'gradients/conv2d_1/kernel.gradient', 'gradients/dense/kernel.gradient',
       'epoch', '_step', 'acc', 'gradients/conv2d_1/bias.gradient',
       'gradients/batch_normalization_4/gamma.gradient', 'loss',
       'gradients/conv2d_2/kernel.gradient',
       'gradients/batch_n

In [ ]:
import wandb
api = wandb.Api()
valid = api.sweep("mu_cv_cnn/mu-cv-assignment2/6llkfa0j")
bayes_dropout = api.sweep("mu_cv_cnn/mu-cv-assignment2/r588c1ll")
bayes= api.sweep("mu_cv_cnn/mu-cv-assignment2/9k18s8pr")
random = api.sweep("mu_cv_cnn/mu-cv-assignment2/zjl5q11e")


In [ ]:
valid.runs

[<Run mu_cv_cnn/mu-cv-assignment2/p71wud18 (finished)>,
 <Run mu_cv_cnn/mu-cv-assignment2/5s5iphfl (finished)>]

In [ ]:

summary_list = [] 
config_list = [] 
name_list = [] 
for run in valid.runs: 
    # run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.  We remove special values that start with _.
    config_list.append({k:v for k,v in run.config.items()}) 

    # run.name is the name of the run.
    name_list.append(run.name)       

import pandas as pd 
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
valid_df = pd.concat([name_df, config_df,summary_df], axis=1)

In [ ]:

summary_list = [] 
config_list = [] 
name_list = [] 
for run in bayes_dropout.runs: 
    # run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.  We remove special values that start with _.
    config_list.append({k:v for k,v in run.config.items()}) 

    # run.name is the name of the run.
    name_list.append(run.name)       

import pandas as pd 
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
bayes_dropout_df = pd.concat([name_df, config_df,summary_df], axis=1)

summary_list = [] 
config_list = [] 
name_list = [] 
for run in bayes.runs: 
    # run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.  We remove special values that start with _.
    config_list.append({k:v for k,v in run.config.items()}) 

    # run.name is the name of the run.
    name_list.append(run.name)       

import pandas as pd 
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
bayes_df = pd.concat([name_df, config_df,summary_df], axis=1)

summary_list = [] 
config_list = [] 
name_list = [] 
for run in random.runs: 
    # run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.  We remove special values that start with _.
    config_list.append({k:v for k,v in run.config.items()}) 

    # run.name is the name of the run.
    name_list.append(run.name)       

import pandas as pd 
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
random_df = pd.concat([name_df, config_df,summary_df], axis=1)

In [ ]:
bayes_df["sweep"]="bayes"
random_df["sweep"]="random"

In [ ]:
 frames = [bayes_df, random_df]

result = pd.concat(frames)

In [ ]:
result["sweep"]

0      bayes
1      bayes
2      bayes
3      bayes
4      bayes
5      bayes
6      bayes
7      bayes
8      bayes
9      bayes
10     bayes
11     bayes
12     bayes
13     bayes
14     bayes
0     random
1     random
2     random
3     random
4     random
5     random
6     random
7     random
8     random
9     random
10    random
11    random
12    random
13    random
14    random
Name: sweep, dtype: object

In [ ]:
import plotly.express as px
fig = px.scatter(result, x=result.index, y="val_acc", color="sweep",
                 hover_data=['dropout',
                             'layer_1_filter',
                             'layer_2_filter',
                             'layer_3_filter',
                             'layer_4_filter',
                             'batchnorm',], template="plotly_white",
                 labels={
                     "val_acc": "Validation accuracy",
                     "x": "# run"
                 })
fig.show()

In [ ]:
wandb.log({"chart": fig})